# Isolation Levels and *(Uselessly)* Concurrent Computation of Fibonacci Numbers 

## 1. Isolation Levels

### 1.1. Introduction
WEIYILEI A0276571W

Isolation in databases means preventing concurrent transactions from interfering and ensuring data consistency and integrity as if the transactions were executed one after the other.

The purpose of serializability levels in database management systems, in general, and in PostgreSQL, in particular, is to tune the degree of isolation between concurrent transactions in search of a trade-off between consistency, isolation, and performance. 

Most database management systems support four isolation levels: read uncommitted, read committed, repeatable read, and serialisable.

### 1.2. Isolation Levels in PostgreSQL

PostgreSQL allows the declaration of each of them but only implements three.

In theory, the read uncommitted level allows a transaction to see changes made by other transactions even before those changes are committed, potentially leading to dirty reads. In PostgreSQL, however, it is implemented at a read committed level.

The read committed level allows a transaction to see only data committed before the transaction begins. A transaction can also see changes made by other transactions that commit while the transaction is still in progress. This level prevents dirty reads, but not non-repeatable reads or phantom reads.

The repeatable read level transaction operates with a snapshot of the database taken at the start of the transaction. It prevents dirty and non-repeatable reads, ensuring that all reads within a transaction see a consistent database snapshot. However, it does not always prevent phantom reads.

The serialisable level This is the highest isolation level and provides strict serializability. It acts as if transactions are executed serially, one after the other, rather than concurrently. This level prevents dirty, non-repeatable, and phantom reads, ensuring total isolation from other transactions.

Choosing the wrong isolation level may improve throughput but compromise data integrity. At higher isolation levels,  the database management system may abort and roll back some transactions to prevent anomalies, potentially reducing concurrency. 

We use the ```psycopg2``` library.

In [1]:
import psycopg2

In psql or pgAdmin4, create a database.

Set the following postgres database, user, password, and port number, accordingly.

In [2]:
database = "asg2"
user = "postgres"
password = "weiyilei0708"
port = "5432"

## 2. Computing Fibonacci Numbers

### 2.1 A Table of Fibonacci Numbers

We propose to program concurrent transactions compute the sequence of Fibonacci numbers by completing a shared table.

Let us create a table ```fibonacci``` and insert the first, 0, and second, 1, Fibonacci numbers and their rank in the respective columns. The ```transaction``` column shall later indicate which transaction has calculated the corresponding entry. The two initial values are given and not calculated.

In [3]:
# Connect to the database
conn = psycopg2.connect(database=database, user=user, password=password, host="localhost", port=port)
# The proper way to begin, end (commit or rollback) a transaction and open and close a cursor in psycopg2  
# is to use in use a connection as a context manager (using the with statement.)
# The begin and commit or rollback of the transaction, open and close of the cursor are implicit.
with conn:
    with conn.cursor() as cur:
        cur.execute("CREATE TABLE IF NOT EXISTS fibonacci (rank SERIAL, number NUMERIC, transaction INT); \
        ALTER SEQUENCE fibonacci_rank_seq RESTART WITH 1; \
        DELETE FROM fibonacci; \
        INSERT INTO fibonacci (number, transaction) VALUES (0, null), (1, null);")
        cur.execute("SELECT * FROM fibonacci ORDER BY number ASC;")
        rows = cur.fetchall()

for rank, number, transaction in rows:
    print(rank, number, transaction)
    
conn.close()

1 0 None
2 1 None


### 2.2. Preparing the Threads and Concurrent Transactions

We use the ```threading``` library.

In [4]:
import threading

In [5]:
def fibonacci(transaction_number):
 try:
    # The thread connects to the database
    conn = psycopg2.connect(database=database, user=user, password=password, host="localhost", port=port)
    
    ###################################################
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE)
    ###################################################
    
    cur = conn.cursor()
    # A transaction starts
    cur.execute("BEGIN;") 
    
    # The transaction finds the highest ranked Fibonacci number already computed  
    cur.execute("SELECT f.rank, f.number FROM fibonacci f \
    WHERE f.rank >= ALL (SELECT f1.rank FROM fibonacci f1);")
    row = cur.fetchone()
    rank_highest = row[0]
    fib_highest = row[1]
    
    # Depending on your machine the threads may run sequentially, 
    # in which case you may want to introduce a random delay to run them concurrently 
    # and observe the anomalies at lower isolation levels
    import random
    import time
    time.sleep(random.randint(0, 10))
    
    # The transaction finds the next highest ranked Fibonacci element. 
    # It is not guaranteed to be the highest ranked minus one as the transactions may not be properly serialised. 
    cur.execute("SELECT f.rank, f.number  FROM fibonacci f \
            WHERE f.rank >= ALL (SELECT f1.rank FROM fibonacci f1 \
                WHERE f1.rank < ANY (SELECT f2.rank FROM fibonacci f2)) \
                AND f.rank < ANY (SELECT f3.rank FROM fibonacci f3);")
    row = cur.fetchone()
    rank_second_highest = row[0]
    fib_second_highest = row[1] 
    
    # Add the newly computed Fibonacci number with the next rank.
    # We do not use the serial sequence to generate the rank 
    # as it does not behave according to the isolation level (you can try to do so)
    cur.execute("INSERT INTO fibonacci (rank, number, transaction) (SELECT "
                + str(rank_highest + 1)+ ", "
                + str(fib_highest + fib_second_highest) 
                + ", " + str(transaction_number)
                + ");")
    conn.commit()
    
 except psycopg2.Error as e:
    #print("\nTransaction " + str(transaction_number) + " aborted and rolled back: ", e.pgerror)
    if conn is not None:
        conn.rollback()
        conn.close()
          
 finally:
    if cur is not None and not cur.closed:
        cur.close()
    if conn is not None and not conn.closed:
        conn.close()

### 2.3. Running the Threads

We create and start *n* threads.

In [6]:
threads = []
n = 99

for transaction_number in range(n):
    thread = threading.Thread(target=fibonacci, args=(transaction_number,))
    threads.append(thread)

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

### 2.4. The Results

We finally retrieve the result computed when the concurrent transactions. Some transactions may have been committed and others aborted and rolled back. All threads have ended.

We print the content of the table *fibonacci*.

In [7]:
# Connect to the database
conn = psycopg2.connect(database=database, user=user, password=password, host="localhost", port=port)
with conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM fibonacci ORDER BY rank ASC;")
        rows = cur.fetchall()
        
conn.close()
        
for rank, number, transaction in rows:
    print("Fibonacci number number ", rank, " is ", number, " and was calculated by transaction number ", transaction)

Fibonacci number number  1  is  0  and was calculated by transaction number  None
Fibonacci number number  2  is  1  and was calculated by transaction number  None
Fibonacci number number  3  is  1  and was calculated by transaction number  7
Fibonacci number number  4  is  2  and was calculated by transaction number  14
Fibonacci number number  5  is  3  and was calculated by transaction number  56
Fibonacci number number  6  is  5  and was calculated by transaction number  62
Fibonacci number number  7  is  8  and was calculated by transaction number  63
Fibonacci number number  8  is  13  and was calculated by transaction number  53
Fibonacci number number  9  is  21  and was calculated by transaction number  48
Fibonacci number number  10  is  34  and was calculated by transaction number  66
Fibonacci number number  11  is  55  and was calculated by transaction number  82
Fibonacci number number  12  is  89  and was calculated by transaction number  74
Fibonacci number number  13  

### 2.5 Analysis

The final output may not match a correct Fibonacci sequence. For instance, the result may contain duplicates sequence elements or wrong Fibonnaci numbers. To observe such issues, run the code several times. The threads may be scheduled and allocated resources differently at each execution.

You may have noticed that the only way for the transactions, as they are programmed, to compute a correct sequence is to run sequentially.

# 3. Questions

## 3.1. Question 1

In the given code, the isolation level is implicit. Call the method that sets the isolation level explicitly in the space indicated. After testing the code, cut the function call out of the main code and paste it below. 

```conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_READ_COMMITTED)```

## 3.1. Question 2

Set the isolation level so that the final sequence is correct although it may contain duplicate sequence elements. Call the method that sets the isolation level accordingly in the space indicated. After testing the code, cut the function call out of the main code and paste it below. 

```conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ)```

## 3.1. Question 3

Set the isolation level so that the final sequence is correct. Call the method that sets the isolation level accordingly in the space indicated. After testing the code, leave the function call in the main code and copy it below. 

```conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE)```

# Appendices

The following are some possibly useful but not necessary SQL commands.

You can kill all connections to PostgreSQL with following SQL code (run it for psql or pgAdmin 4.)

```
SELECT 
    pg_terminate_backend(pid) 
FROM 
    pg_stat_activity 
WHERE 
    pid <> pg_backend_pid()
    AND datname = '<database_name>';
```
 
You can check the maximum number of connections to PostgreSQL with following SQL code.

```
SHOW MAX_CONNECTIONS;
```

You can change the maximum number of connections to PostgreSQL with following SQL code. You will need to restart you PostgreSQL server afterwards.

```
ALTER SYSTEM SET max_connections = <num_of_connections>;
```